In [50]:
from pynq import Overlay
from pynq.lib.pmod import pmod_adc
import time
from pynq import MMIO

# Load the SPI overlay
ol = Overlay("spi.bit")
spi = ol.axi_quad_spi_0

In [51]:
# Access the IP dictionary to find the address of the axi_quad_spi_0
spi_ip = ol.ip_dict['axi_quad_spi_0']
spi_base_address = (spi_ip['phys_addr'])
spi_address_range = (spi_ip['addr_range']) 


In [52]:
# SPI initialization function
def init(spi):
    print("Initializing SPI...")
    spi.write(0x40, 0x0a)       # Configure the SPI
    spi.write(0x28, 0x04)
    spi.write(0x1c, 0x00)
    spi.write(0x70, 0xFFFFFFFF)
    
    ctrlreg = spi.read(0x60)    # Read control register
    print(f"Initial Control Register (0x60): {hex(ctrlreg)}")
    
    ctrlreg = ctrlreg | 0xe6    # Modify control register
    spi.write(0x60, ctrlreg)
    
    ctrlreg = spi.read(0x60)    # Ensure the control register is correctly set
    print(f"Modified Control Register (0x60): {hex(ctrlreg)}")
    
    ctrlreg = ctrlreg & ~(0x15)
    spi.write(0x60, ctrlreg)
    
    print("SPI Initialized.")

In [53]:
# SPI transfer function with timeout
def transfer(packet, spi, timeout=1):
    for data in packet:
        print(f"Writing data: {hex(data)} to Tx FIFO")
        spi.write(0x68, data)   # Write data to Tx FIFO
        spi.write(0x70, 0xFFFFFFFE)   # Start transfer
        
        ctrlreg = spi.read(0x60)
        ctrlreg = ctrlreg & ~(0x100)  # Clear the transfer complete flag
        spi.write(0x60, ctrlreg)
        
        # Wait for transfer to complete with a timeout
        start_time = time.time()
        statReg = spi.read(0x64)
        while (statReg & 0x04) == 0:
            if time.time() - start_time > timeout:
                print("Timeout during transfer")
                return None  # Timeout occurred
            statReg = spi.read(0x64)
        
        print("Transfer completed.")
        
        ctrlreg = spi.read(0x60)
        ctrlreg = ctrlreg | 0x100     # Set the transfer complete flag
        spi.write(0x60, ctrlreg)
    
    spi.write(0x70, 0xFFFFFFFF)       # Reset the SPI interface
    
    recvData = list()
    # Read data from Rx FIFO with a timeout
    start_time = time.time()
    RxFifoStatus = spi.read(0x64) & 0x01
    while RxFifoStatus == 0:
        if time.time() - start_time > timeout:
            print("Timeout while reading Rx FIFO")
            return None  # Timeout occurred
        temp = spi.read(0x6c)         # Read data from Rx FIFO
        recvData.append(temp)
        RxFifoStatus = spi.read(0x64) & 0x01
    
    print(f"Received data: {recvData}")
    return recvData


In [54]:
# Main test program for a single data point transfer
init(spi)
packet = [0x00]  # Adjust the packet content as per your SPI device
print("Testing single packet transfer...")
data = transfer(packet, spi)

if data is not None:
    print(f"Data received: {data}")
else:
    print("No data received or transfer error.")

Initializing SPI...
Initial Control Register (0x60): 0x180
Modified Control Register (0x60): 0x186
SPI Initialized.
Testing single packet transfer...
Writing data: 0x0 to Tx FIFO
Timeout during transfer
No data received or transfer error.
